#  Patrick's "Applied Data Science" Coursera-Capstone Project!!

This Notebook Document's Patrick's "Applied Data Science" Coursera-Capstone Project!!

In [132]:
import pandas as pd
import numpy as np

In [133]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## The Following Section begins the Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

In [134]:
import json # library to handle JSON files
#from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium

# Import BeautifulSoup package to parse wiki website
from bs4 import BeautifulSoup

#### Begin Scraping the Wiki Page for the Toronto Postal Codes, go to URL and Find the Table

In [135]:
website_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(website_url,'lxml')
PC_table = soup.find("table")

#### Parse Extracted Table for text entries, then split text into postal code, borough, and neighborhood and append findings to datframe (ignoring entries with "Not assigned" for Borough)

In [136]:
df=pd.DataFrame(columns=['Postal Code', 'Borough', 'Neighborhood'])  # Initialize empty dataframe

for link in PC_table.find_all("td"):
    Link_Text = str(link.text.replace('\n', ' ').strip())  #strip out carriage returns
    postal_code=Link_Text[0:3]
    if postal_code.startswith('M')==1:
        B1=Link_Text[3:]
        if B1.startswith('Not')==0:   # Skip entries that begin with "Not assigned"
            if B1.find("(")!=-1:
                X=B1.split('(')
                borough=X[0]
                X2=X[1].replace(')'," ")
                X2=X2.replace('/',",")
                neighborhood=X2
            else:
                borough=B1.replace('/',",")
                neighborhood=B1.replace('/',",")
            
            df = df.append({'Postal Code': postal_code, 'Borough': borough, 'Neighborhood': neighborhood}, ignore_index=True)
    
df.head(12)

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park , Harbourfront"
3,M6A,North York,"Lawrence Manor , Lawrence Heights"
4,M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government"
5,M9A,Etobicoke,Islington Avenue
6,M1B,Scarborough,"Malvern , Rouge"
7,M3B,North York,Don Mills North
8,M4B,East York,"Parkview Hill , Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [137]:
df.shape

(103, 3)

## Begin the second project of the Assignment

In [138]:
import geopy 
from geopy.geocoders import Nominatim
import geocoder

#### I attempted the "geocoder" method, but after several attempts with no results I gave up.

In [139]:
import geocoder # import geocoder

# initialize your variable to None
#lat_lng_coords = None

#result = [] 
#for value in df["PostalCode"]: 
#    postal_code=value
    # loop until you get the coordinates
#    while(lat_lng_coords is None):
#        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
#        lat_lng_coords = g.latlng
#        latitude = lat_lng_coords[0]
#        longitude = lat_lng_coords[1]
#        df = df.append({'Latitude': latitude, 'Longitude': longitude})    

#df.head()                    

#### I even tried to get it to work once manually, and after hitting "ctrl+enter" about 30 or 40 times with no results, I decided it wasn't going to work for me...

In [140]:
g = geocoder.google('{}, Toronto, Ontario'.format("M1B"))
lat_lng_coords = g.latlng
print(lat_lng_coords)

None


#### Import the Postal Code .csv file and merge with Neighborhood postal codes from above.

In [141]:
filename="Geospatial_Coordinates.csv"
df2=pd.read_csv(filename)
df3 = pd.merge(df, df2, on="Postal Code")
df3.head(12)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763
4,M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Malvern , Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Parkview Hill , Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


## Begin the third project of the Assignment

### First, map all the neighborhood data without clustering to get a feel for it.

In [191]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="To_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [198]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df3['Latitude'], df3['Longitude'], df3['Borough'], df3['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### To begin the neighborhood analysis part of the project, begin by Defining the FourSquare Credentials

In [142]:
CLIENT_ID = 'G4XYV2OWUJSPBPVI00SJXGVWFAP21E3ZNJXHWTSAODQPKIVG' # your Foursquare ID
CLIENT_SECRET = 'J3TXUHL4AZBXEYYRU2OF4OW33BXUR1KKSCMWHLJOKYQUJK2I' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: G4XYV2OWUJSPBPVI00SJXGVWFAP21E3ZNJXHWTSAODQPKIVG
CLIENT_SECRET:J3TXUHL4AZBXEYYRU2OF4OW33BXUR1KKSCMWHLJOKYQUJK2I


#### Use the same function from the New York project to process all the neighborhoods in Toronto

In [143]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    radius = 500 # define radius
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        #print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

### Use the Above Definition to get FourSquare Venue Info from all of the Toronto Neighborhoods


In [144]:
Toronto_venues = getNearbyVenues(names=df3['Neighborhood'],
                                   latitudes=df3['Latitude'],
                                   longitudes=df3['Longitude']
                                  )

In [145]:
print(Toronto_venues.shape)
Toronto_venues.head()

(2225, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
2,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
3,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop
4,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant


#### After the neighborhood grouping, we have fewer rows than in the initial table above, so there must be some neighborhoods that span postal codes

In [146]:
Toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Agincourt,4,4,4,4,4,4
"Alderwood , Long Branch",9,9,9,9,9,9
"Bathurst Manor , Wilson Heights , Downsview North",20,20,20,20,20,20
Bayview Village,4,4,4,4,4,4
"Bedford Park , Lawrence Manor East",23,23,23,23,23,23
...,...,...,...,...,...,...
Willowdale West,7,7,7,7,7,7
Woburn,4,4,4,4,4,4
Woodbine Heights,8,8,8,8,8,8


In [147]:
print('There are {} uniques categories.'.format(len(Toronto_venues['Venue Category'].unique())))

There are 268 uniques categories.


#### Convert Neighborhood data to onehot encoding

In [148]:
# one hot encoding
Toronto_onehot = pd.get_dummies(Toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Toronto_onehot['Neighborhood'] = Toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Toronto_onehot.columns[-1]] + list(Toronto_onehot.columns[:-1])
Toronto_onehot = Toronto_onehot[fixed_columns]

Toronto_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [149]:
Toronto_onehot.shape

(2225, 268)

In [150]:
Toronto_grouped = Toronto_onehot.groupby('Neighborhood').mean().reset_index()
Toronto_grouped.head()

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Warehouse Store,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,"Alderwood , Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,"Bathurst Manor , Wilson Heights , Downsview No...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,"Bedford Park , Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.043478,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [151]:
Toronto_grouped.shape

(99, 268)

### Identify the most common venues in each Toronto Neighborhood, and output dataframe grouped by neighborhood.

In [189]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [190]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Toronto_grouped['Neighborhood']

for ind in np.arange(Toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Latin American Restaurant,Skating Rink,Breakfast Spot,Lounge,Donut Shop,Diner,Discount Store,Distribution Center,Dog Run,Doner Restaurant
1,"Alderwood , Long Branch",Pizza Place,Gym,Athletics & Sports,Pharmacy,Pub,Sandwich Place,Skating Rink,Coffee Shop,Airport Terminal,American Restaurant
2,"Bathurst Manor , Wilson Heights , Downsview No...",Bank,Coffee Shop,Frozen Yogurt Shop,Pharmacy,Supermarket,Sushi Restaurant,Middle Eastern Restaurant,Diner,Restaurant,Pizza Place
3,Bayview Village,Café,Bank,Chinese Restaurant,Japanese Restaurant,Doner Restaurant,Diner,Discount Store,Distribution Center,Dog Run,Donut Shop
4,"Bedford Park , Lawrence Manor East",Coffee Shop,Restaurant,Italian Restaurant,Sandwich Place,Pharmacy,Comfort Food Restaurant,Butcher,Indian Restaurant,Café,Greek Restaurant


### Begin K-Cluster Analysis of Neighborhood Venues, Chose 4 clusters because 5 did not look like much

In [183]:
# set number of clusters
kclusters = 4

Toronto_grouped_clustering = Toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 2])

In [184]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

Toronto_merged = df3

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
Toronto_merged = Toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

Toronto_merged.head() # check the last columns!

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,2.0,Park,Food & Drink Shop,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Doner Restaurant,Event Space
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Portuguese Restaurant,Intersection,Hockey Arena,Women's Store,Dog Run,Dim Sum Restaurant,Diner,Discount Store,Distribution Center
2,M5A,Downtown Toronto,"Regent Park , Harbourfront",43.654260,-79.360636,1.0,Coffee Shop,Pub,Park,Bakery,Theater,Mexican Restaurant,Café,Hotel,Spa,Ice Cream Shop
3,M6A,North York,"Lawrence Manor , Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Event Space,Vietnamese Restaurant,Furniture / Home Store,Coffee Shop,Miscellaneous Shop,Boutique,Accessories Store,Eastern European Restaurant,Electronics Store
4,M7A,"Queen's Park , Ontario Provincial Government","Queen's Park , Ontario Provincial Government",43.662301,-79.389494,1.0,Coffee Shop,Park,Gym,Diner,Nightclub,Mexican Restaurant,Juice Bar,Italian Restaurant,Hobby Shop,Wings Joint


#### Not all the neighborhood's returned venue information from FourSquare, so the NAN values needed to be dropped.

In [185]:
Toronto_merged.dropna(subset=['Cluster Labels'],inplace=True)

### The Toronto Cluster Map below shows the Neighborhood Venue Clustering Calculated. There does not appear to be much diversity (Cluster 1 dominates the map). Perhaps limiting it to only the "Toronto" Borough would have provided a smaller, but more diverse clustering result.

In [187]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Toronto_merged['Latitude'], Toronto_merged['Longitude'], Toronto_merged['Neighborhood'], Toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters